In [1]:
import pandas as pd
import torch

from model.encoder import Encoder

from utils.misc import cosine_distance

/home/tdutton/mambaforge/envs/cers/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
movies = pd.read_csv("data/ml-100k/u.item", sep='|', names=["movie_id", "movie_title", "release_date", "dummy", "url", "unknown", "Action", "Adventure", "Animation", "Children's", "Comedy", "Crime", "Documentary", "Drama", "Fantasy", "Film-Noir", "Horror", "Musical", "Mystery", "Romance", "Sci-Fi", "Thriller", "War", "Western"], encoding='latin1', header=None)

movies = movies[["movie_id", "movie_title"]]

In [3]:
device = "cuda" if torch.cuda.is_available() else "cpu"

movie_embeddings = torch.load("weights/recommender/deepfm.pt", map_location="cpu")["embedding.embedding.weight"][943:]

encoder = Encoder().to(device)

encoder.load_state_dict(torch.load("weights/encoder/encoder.pt", map_location=device))

/tmp/ipykernel_33589/1772756124.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  movie_embeddings = torch.load("weights/recommender/deepfm.pt", map_location="cpu")["embed

<All keys matched successfully>

In [6]:
request = "I want to watch a romantic comedy."

request_embedding = encoder(request).cpu()

distances = cosine_distance(request_embedding, movie_embeddings)

_, indices = torch.topk(distances, k=10, largest=False)

movies.iloc[indices]

,movie_id,movie_title
353,354,"Wedding Singer, The (1998)"
254,255,My Best Friend's Wedding (1997)
764,765,Boomerang (1992)
1217,1218,Friday (1995)
110,111,"Truth About Cats & Dogs, The (1996)"
250,251,Shall We Dance? (1996)
215,216,When Harry Met Sally... (1989)
534,535,Addicted to Love (1997)
12,13,Mighty Aphrodite (1995)
475,476,"First Wives Club, The (1996)"
